# US Wildfires
## A Review of the class example

Main library to manage geospatial data: 
- Geopandas https://geopandas.org/index.html

In [ ]:
import numpy
import matplotlib.pyplot as plt
import geopandas
import pyproj
import rasterio
import rasterstats
import os
import json

In [ ]:
os.getcwd()


In [ ]:
data_dir=r"C:\Users\Hanna Willwerth\ACE-592\TA_Material\2_Images"

## Reading  State json file

In [ ]:
state_file = "2_Images/us_states.json"
country = geopandas.read_file(state_file)

In [ ]:
country.head()

In [ ]:
type(country)

In [ ]:
country.crs

In [ ]:
# pip install descartes

In [ ]:
country.plot()


In [ ]:
country = country[~country['NAME'].isin(["Alaska","Hawaii","Puerto Rico"])]

In [ ]:
country.plot();

In [ ]:
f,a = plt.subplots(figsize=(10,7));
country.plot(ax=a,
             column="CENSUSAREA",
             legend=True);
plt.axis("off");

## Reading Wildfires shapefile

In [ ]:
fires = geopandas.read_file(data_dir+"\MODIS_C6_USA_contiguous_and_Hawaii_7d.shp")
fires.head()

In [ ]:
fires.geometry

In [ ]:
fires.plot()

## Plot both together
#### BEFORE! Always check projections

In [ ]:
country.crs == fires.crs

In [ ]:
f,a = plt.subplots(figsize=(15,10))  # Figure and Axis way 
country.plot(ax=a,color="grey") # Plotting the first layer: country  (ax=a means the we want to add that plot to the axis)
fires.plot(color="C3",markersize=10,ax=a)  # Plotting the second layer: Fires
plt.axis("off");  # Removes axis 

## Clipping 
Both layers must be in the same Coordinate Reference System (CRS)

Syntax: `geopandas.clip(gdp, mask)`.
- `gdp` geopandas data frame (points, polygons) to be clipped
- `mask` polygon vector layer used to clip gdf.

In [ ]:
fires_clipped=geopandas.clip(fires,country)

In [ ]:
f,a = plt.subplots(figsize=(15,10))

country.plot(ax=a,color="grey")
fires_clipped.plot(color="C3",markersize=10,ax=a);
plt.axis("off");

In [ ]:
f,a = plt.subplots(figsize=(15,8))

country.plot(ax=a,color="grey")
fires_clipped.plot(markersize=10, # set the markersize 
                    ax=a, # use the current axis we created
                    column='BRIGHTNESS', #column to use
                    cmap="OrRd", # colormap to use
                    #scheme='quantiles', # this keyword breaks the categories on quantiles.  
                    # The 'mapclassify' >= 2.2.0 package is required to use the 'scheme' keyword
                    legend=True); # show a legend

## Spatial operations

### Left vs right spatial joins

Syntax: `geopandas.sjoin(left_df, right_df, how='inner', lsuffix='left', rsuffix='right')`

- `left`: use keys from left_df; retain only left_df geometry column

- `right`: use keys from right_df; retain only right_df geometry column

- `inner`: use intersection of keys from both dfs; retain only left_df geometry column

In [ ]:
fires_clipped.head(3)

In [ ]:
country.head(3)

### Fires with state - left

In [ ]:
fires_w_state = geopandas.sjoin(fires_clipped,\
                                country,\
                                how='left')

In [ ]:
fires_w_state.head(5)

In [ ]:
f,a = plt.subplots();
country.plot(color='gray',ax=a);
fires_w_state.plot(column="CENSUSAREA",ax=a);  #Now we can add here columns that used to be part of country  gpd 
plt.title("Fires Colored by Census Area");

### States with fires - right

In [ ]:
state_w_fires = geopandas.sjoin(fires_clipped,\
                                country,\
                                how='right')


In [ ]:
state_w_fires.head(5)

In [ ]:
state_fire_counts = state_w_fires.groupby("NAME")['index_left'].count().reset_index()

state_fire_counts.columns = ["NAME","fire_number"]

state_fire_counts.head()

In [ ]:
country = country.merge(state_fire_counts,how='left')
country.head()

In [ ]:
f,a = plt.subplots(figsize=(15,8));
country.plot(column="fire_number",
             ax=a,
             cmap="OrRd",
            legend=True);

# Raster files

In [ ]:
#historical weather data
raster_file = "2_Images/PRISM_ppt_stable_4kmM3_2017_bil.bil"
raster = rasterio.open(raster_file)

In [ ]:
raster

In [ ]:
raster.read(1)

In [ ]:
plt.imshow(raster.read(1))

In [ ]:
raster.bounds

In [ ]:
raster.crs

In [ ]:
country.crs

Reprojection:
- Reproject the raster file to match the shapefile (harder).
- Reproject the shapefile to match the raster (easier).

In [ ]:
country_proj = country.to_crs(epsg=4087) # WGS 84 Equidistant Cylndrical (Look how easy is to reproject a geopandas df)
country_proj.crs

In [ ]:
import numpy as np
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import geopandas

src_file = raster_file
dst_file = 'ppt_reproj.tif'
dst_crs = rasterio.crs.CRS.from_dict({'init': 'EPSG:4087'})

In [ ]:
with rasterio.open(src_file) as src: 
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds) 
    kwargs = src.meta.copy() 
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })
    with rasterio.open(dst_file, 'w', **kwargs) as dst: 
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest)

In [ ]:
raster = rasterio.open("./ppt_reproj.tif")


In [ ]:
raster.crs

In [ ]:
raster.crs == country_proj.crs

In [ ]:
country_proj.plot()

In [ ]:
country_proj.head(3)

In [ ]:
json.loads(country_proj[country_proj.NAME=="Illinois"]['geometry'].to_json())['features'][0]['geometry'];

In [ ]:
def getFeatures(gdf):
    """Function to parse features from GeoDataFrame in such a manner that rasterio wants them"""
    import json
    return [json.loads(gdf.to_json())['features'][0]['geometry']]

IL_coords = getFeatures(country_proj[country_proj.NAME=="Illinois"])

In [ ]:
IL_coords;

In [ ]:
coords = np.array(IL_coords[0]['coordinates'][0])
plt.plot(coords[:,0],coords[:,1],color='black')

### Mask from rasterio
Mask the area outside of the input shapes with no data.

Syntax: `rasterio.mask.mask(dataset, shapes, all_touched=False, invert=False, nodata=None, filled=True, crop=False, pad=False, pad_width=0.5, indexes=None)`

- dataset (a dataset object opened in 'r' mode) – Raster to which the mask will be applied.

- shapes (iterable object) – The values must be a GeoJSON-like dict or an object that implements the Python geo interface protocol (such as a Shapely Polygon).

In [ ]:
from rasterio.mask import mask

img_out, img_transform = mask(raster,IL_coords)


In [ ]:
plt.imshow(img_out[0],cmap="Blues")
# So now we have just Illinois.
# But notice now that the indices dont have anything to do with geo-coordinates.

In [ ]:
img_transform
# This is the affine transformation that translate pixel coordinates into geographic coordinates

Notice that the axis in the plot above are not geographic coordinates. 
We need to make the `raster` again using the information from the affine transformation

In [ ]:
out_meta = raster.meta.copy()

out_meta['height'] = img_out.shape[1]
out_meta['width'] = img_out.shape[2]
out_meta['transform'] = img_transform

with rasterio.open("./IL_PPT.tif", "w", **out_meta) as dest:
    dest.write(img_out)

In [ ]:
IL_raster = rasterio.open("./IL_PPT.tif")

In [ ]:
from rasterio.plot import show

fig, ax = plt.subplots(figsize=(15, 15))
rasterio.plot.show(IL_raster, ax=ax,cmap="Blues")
country_proj.plot(ax=ax, facecolor='none', edgecolor='black')